# Data Modeling 03: Deep Learning

In this notebook, we were begin to explore deep learning as a possibility for this competition. Although deep learning models like RNN's and LSTM's are able to capture information about the past, they are very susceptible to overfitting. Since our dataset is relatively small, we will most likely have to introduce significant regularization to prevent overfitting. 

In [35]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
df = pd.read_csv('../data/clean/full/dengue_features_train.csv')
df_labels = pd.read_csv('../data/clean/full/dengue_labels_train.csv')
df_test = pd.read_csv('../data/raw/dengue_features_test.csv')

sj_features = pd.read_csv('../data/clean/sj/sj_train_features.csv')
sj_labels = pd.read_csv('../data/clean/sj/sj_train_labels.csv')

iq_features = pd.read_csv('../data/clean/iq/iq_train_features.csv')
iq_labels = pd.read_csv('../data/clean/iq/iq_train_labels.csv')

sj_test = pd.read_csv('../data/clean/sj/sj_test_features.csv')
iq_test = pd.read_csv('../data/clean/iq/iq_test_features.csv')

In [4]:
sj_features.head(2)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_max_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.1226,0.103725,0.198483,0.177617,297.572857,297.742857,...,299.8,32.00,73.365714,12.42,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.1699,0.142175,0.162357,0.155486,298.211429,298.442857,...,300.9,17.94,77.368571,22.82,2.371429,26.714286,6.371429,31.7,22.2,8.6


Regularized deep learning

In [45]:
def trained_model(df, df_labels):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(['city', 'week_start_date'], axis=1), df_labels['total_cases'])

    model = keras.Sequential([
        layers.BatchNormalization(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])

    model.compile(loss='mae',
                optimizer='adam',
                metrics=['mae'])

    hist = model.fit(
        X_train, y_train,
        epochs=1000,
        callbacks=[keras.callbacks.EarlyStopping(patience=40, monitor='mae')],
        verbose=0
    )
    
    return model

sj_m = trained_model(sj_features, sj_labels)
iq_m = trained_model(iq_features, iq_labels)



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [49]:
sj_p = sj_m.predict(sj_test.drop(['city', 'week_start_date'], axis=1))
iq_p = iq_m.predict(iq_test.drop(['city', 'week_start_date'], axis=1))
pred = np.concatenate((sj_p, iq_p))
subm = pd.read_csv('../data/raw/submission_format.csv')
subm['total_cases'] = pred.astype(int)
subm.to_csv('../subm6.csv', index=False)